Import all librairies

In [ ]:
import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import butter, filtfilt, freqz
import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd
#MNE
import mne
from mne.time_frequency import tfr_morlet
from importlib import reload 
from os.path import join, dirname, realpath, exists

#load poly5reader
# define current working directory (the one that contains this notebook): this is the project_path
os.chdir('c:\\Users\\Juliette\\Research\\Projects\\Synchronization project\\Code\\Synchronization')
project_path = os.getcwd()

# change temporarily the working directory to go fetch the poly5 reader: this is the code_path
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(os.path.join(os.getcwd(), 'tmsi-python-interface-main'))
code_path = os.getcwd()

print (f'working dir for analysis:{project_path}')
print (f'working dir to go fetch Poly5 reader:{code_path}')

#fetch the poly5 reader
from TMSiFileFormats.file_readers import Poly5Reader

# set the working directory back to the project_path
os.chdir(project_path)

# load pyPerceive functions
os.chdir('c:\\Users\\Juliette\\Research\\Projects\\Synchronization project\\Code\\Synchronization')
project_path = os.getcwd()
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(os.path.join(os.getcwd(), 'PyPerceive'))
os.chdir(os.path.join(os.getcwd(), 'code'))
code_path = os.getcwd()
print (f'working dir for analysis:{project_path}')
print (f'working dir to go fetch PyPerceive functions:{code_path}')

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

#reset the proper working directory for the analysis
os.chdir(project_path)

Open Percept file

In [ ]:
#change datapath, filename, name of the variable containing the array + index of the channel if necessary

datapath = 'C:\\Users\\Juliette\\OneDrive - Charité - Universitätsmedizin Berlin\\Recordings\\PERCEPT\\sub-029\\FTG\\sub-000PStn\\ses-2023021610075182\\ieeg'
filename = 'sub-000PStn_ses-2023021610075182_run-BrainSense20230216104500.mat'
data = mne.io.read_raw_fieldtrip(        
	os.path.join(datapath, filename),        
	info={}, # add info here        
	data_name='data',  # name of heading dict/ variable of original MATLAB object    
)
data.ch_names
n_chan = len(data.ch_names)
data.n_times #number of samples
data.times #timepoints set to zero
data._data # channel trials
data.info['sfreq'] # sampling frequency
time_duration = (data.n_times/data.info['sfreq']).astype(float)
time_duration_ms=time_duration*1000
print(     
	f'The data object has:\n\t{data.n_times} time samples,'      
	f'\n\tand a sample frequency of {data.info["sfreq"]} Hz'      
	f'\n\twith a recording duration of {time_duration} seconds.'      
	f'\n\t{n_chan} channels were labeled as \n{data.ch_names}.')

LFP_Stn_R_13 = data.get_data()[1]
x1_time=np.arange(0,time_duration_ms,time_duration_ms/data.n_times)

#visualize raw datas
plt.xlabel('Time (ms)')
plt.ylabel('LFP (µV)')
plt.plot(x1_time,LFP_Stn_R_13, linewidth=1,color='orange')


Open TMSi file

In [ ]:
#change name of the variable containing the array + index of the channel if necessary

TMSi_data = Poly5Reader()

# Extract the samples and channel names from the Poly5Reader object
samples = TMSi_data.samples
ch_names = TMSi_data.ch_names

# Conversion to MNE raw array

toMNE = True
mne_object = TMSi_data.read_data_MNE()

# Retrieve the MNE RawArray info and sample data
info_mne = mne_object.info
samples_mne = mne_object._data
    
#mne_object.plot(start = 5, duration = 5, n_channels = 2) #(only if I want to plot in interactive viewer)

n_chan = len(mne_object.ch_names)
time_duration = (mne_object.n_times/mne_object.info['sfreq']).astype(float)
time_duration_ms=time_duration*1000

print(     
	f'The data object has:\n\t{mne_object.n_times} time samples,'      
	f'\n\tand a sample frequency of {mne_object.info["sfreq"]} Hz'      
	f'\n\twith a recording duration of {time_duration} seconds.'      
	f'\n\t{n_chan} channels were labeled as \n{mne_object.ch_names}.')

BIP01 = mne_object.get_data()[0]
BIP02 = mne_object.get_data()[1]
BIP03 = mne_object.get_data()[2]
x2_time=np.arange(0,time_duration_ms,time_duration_ms/mne_object.n_times)

#visualize raw data
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (AU)')
plt.plot(x2_time,BIP03,linewidth=1)
plt.xlim
plt.show()

mne_object.plot(start = 5, duration = 5, n_channels = 3)

Plot in the same graph TMSi + Percept datas

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1)
fig.suptitle('Sub-029_FTG_M1S1_Ent_RampUpThresh_artefact1') #change title
ax1.ticklabel_format(axis='x',style='sci',scilimits=(3,3))
ax1.axes.xaxis.set_ticklabels([])
ax2.ticklabel_format(axis='x',style='sci',scilimits=(3,3))
ax2.axes.xaxis.set_ticklabels([])
ax3.ticklabel_format(axis='x',style='sci',scilimits=(3,3))
ax3.axes.xaxis.set_ticklabels([])
ax4.ticklabel_format(axis='x',style='sci',scilimits=(3,3))
ax4.set_xlabel('Time (ms)')
ax2.set_ylabel('BIP01\nIPG + Cable',size=10)
ax3.set_ylabel('BIP02\nAcross IPG',size=10)
ax4.set_ylabel('BIP03\nECG',size=10)
ax1.set_ylabel('LFP (µV)',size=10)
ax1.set_xlim(156000,160000)
ax2.set_xlim(171700,175700)
#ax2.set_ylim(0,0.005)
ax3.set_xlim(171700,175700)
ax3.set_ylim(-0.004,0.004)
ax4.set_xlim(171700,175700)
ax4.set_ylim(-0.03,-0.025)
ax1.plot(x1_time,LFP_Stn_R_13,color='orange') #select proper LFP channel
ax2.plot(x2_time,BIP01, color='blue') #select proper sensor channel
ax3.plot(x2_time,BIP02, color='green') #select proper sensor channel
ax4.plot(x2_time,BIP03, color='purple') #select proper sensor channel

Save the plot in the Figures file

In [ ]:
fig.savefig('C:\\Users\\Juliette\\OneDrive\\Documents\\PhD\\Figures\\Sub-023_20MFU_M1S0_BrStr_Rest_artefact6.png',transparent=True, bbox_inches='tight')